Часть 1


Задание

3. Реализовать один из методов векторизации текста и обучить одну из моделей классического ML. Можно использовать любые библитеки
4. Предложить метрику, разбить обучающую выборку на обучающую и валидационную и посчитать ее значение для методов, реализованных в данном ноутбуке, и наивного метода(реализован в ноутбуке "Часть 1 (задания 1,2)")

#Импорт библиотек


In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.svm import SVC

# Загрузка и чтение данных

In [ ]:
# Читаем подготовленные данные
# Предобработка выполнена ранее (ноутбук "Часть 1 (задания 1 и 2)")

train = pd.read_csv('/content/df.csv').dropna()
train.drop(train.columns[0], axis=1, inplace=True)
train

,text,label
0,and and,Neutral
1,advice talk to your neighbours family to excha...,Positive
2,coronavirus australia woolworths to give elder...,Positive
3,my food stock is not the only one which is emp...,Positive
4,me ready to go at supermarket during the covid...,Extremely Negative
...,...,...
41150,airline pilots offering to stock supermarket s...,Neutral
41151,response to complaint not provided citing covi...,Extremely Negative
41152,you know it s getting tough when is rationing ...,Positive
41153,is it wrong that the smell of hand sanitizer i...,Neutral


#Формирование обучающей и валидационной выборок

In [ ]:
# Посмотрим распределение классов
# Имеем 5 классов, классы несбалансированы

train.label.value_counts()

Positive              11422
Negative               9917
Neutral                7711
Extremely Positive     6624
Extremely Negative     5481
Name: label, dtype: int64

In [ ]:
# Подсчитаем количество категорий для классификации

num_classes = len(train.label.value_counts())
print('Количество категорий для классификации: {}'.format(num_classes))

Количество категорий для классификации: 5


In [ ]:
# Переведем метки классов в диапазон значение 0-4

LABEL_TO_ID = {'Extremely Negative': 0, 'Negative': 1,  'Neutral': 2,
               'Positive': 3, 'Extremely Positive': 4}
train.replace({'label': LABEL_TO_ID}, inplace=True)

In [ ]:
# Разбиваем выборку на тренировочную и валидационную

X_train, X_val, y_train, y_val = train_test_split(train.text,
                                                  train.label,
                                                  test_size=0.2,
                                                  random_state=42
                                                  )

In [ ]:
# Проверим размерность получившихся выборок

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((32924,), (32924,), (8231,), (8231,))

#Метод векторизации текстов TF-IDF


Извлечение функций из текстовых файлов

In [ ]:
# Преобразуем текстовые файлы в числовые векторы функций

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(32924, 40905)

In [ ]:
# Методом TF-IDF получаем эмбеддинги

tfidf_transformer = TfidfTransformer()

##Классификация методом MultinomialNB

In [ ]:
# Построим конвейер операций

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(X_train, y_train)

In [ ]:
# Получим предсказание модели

predicted = text_clf.predict(X_val)
np.mean(predicted == y_val)

0.36824201190620826

In [ ]:
# Выведем метрики классификации

print(metrics.classification_report(predicted, y_val))

              precision    recall  f1-score   support

           0       0.01      0.83      0.03        18
           1       0.40      0.40      0.40      1999
           2       0.08      0.80      0.15       156
           3       0.90      0.34      0.50      6027
           4       0.02      0.84      0.04        31

    accuracy                           0.37      8231
   macro avg       0.28      0.64      0.22      8231
weighted avg       0.76      0.37      0.46      8231



Получили неудовлетворительные результаты.

##Классификация методом SGDClassifier

In [ ]:
# Training Support Vector Machines - SVM and calculating its performance

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                                   alpha=1e-3, max_iter=5,
                                                   random_state=42))])

text_clf_svm = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_val)
np.mean(predicted_svm == y_val)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


0.5355363868302758

In [ ]:
# Выведем метрики классификации

print(metrics.classification_report(predicted_svm, y_val))

              precision    recall  f1-score   support

           0       0.75      0.51      0.61      1583
           1       0.37      0.51      0.43      1416
           2       0.72      0.57      0.64      1943
           3       0.31      0.55      0.39      1294
           4       0.79      0.53      0.64      1995

    accuracy                           0.54      8231
   macro avg       0.59      0.53      0.54      8231
weighted avg       0.62      0.54      0.56      8231



#Применяемые метрики

В связи с тем, что датасет не сбалансирован, метрику «accuracy» (доля верных ответов) использовать нецелесообразно, так как  может привести к завышенной оценке качества работы классификатора. В данном случае самое оптимальное – считать сразу несколько метрик, устойчивых к распределению классов (в данном случае, это — точность, полнота и f-мера) и смотреть на них все. Если нам удобно получить одно число, по которому можно понять, насколько хорошо модель работает, то к нашей задаче лучше всего подходит «macro-avg» (сначала подсчитывается каждая метрика по каждому классу, а потом усредняется). Macro-avg более устойчива к скошенным распределениям классов.

Показатель accuracy приведен справочно.

#Выводы

 Применили 2 метода классификации текстов.

 Метод классификации текста SGDClassifier показал лучшие результаты по сравнению с методом MultinomialNB:


*   accuracy составила 0.54 против 0.37;

*   precision - 0.59 против 0.28;


*   recall - 0.53 против 0.64 (единственный показатель метода SGDClassifier, уступающий методу MultinomialNB)
*   f1-score - 0.54 против 0.22


Модель лучше предсказывает более многочисленные классы (f1-score по ним составляет 0.61-0.64), f1-score по остальным классам составила 0.39-0.43.


Модель SGDClassifier лучше справилась с задачей классификации текстов по сравнению с наивными методами (реализованы без использования ML), отраженными в блокноте "Часть 1 (задачи 1 и 2)".
